In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings(action='ignore')

# 데이터 로드

In [2]:
ticker_df = pd.read_csv('../데이터/최종기업ticker.csv', names = ['names','ticker'])
ticker_df = ticker_df.dropna()

In [3]:
df_list =[]
for i in range(len(ticker_df)):
    name = ticker_df['names'][i+1]
    df_i = pd.read_excel(f'../데이터/공시정보/공시정보_2010~2017/{name}.xlsx', header=None)
    # df_i = df_i.drop(['1'], axis=1)
    df_list.append(df_i)
    

In [4]:
df_list[0].head(2)

,0,1,2,3,4,5
0,1,코 CMG제약 IR,주주명부폐쇄기간또는기준일설정,CMG제약,2017.12.12,코
1,2,코 CMG제약 IR,임원ㆍ주요주주특정증권등소유상황보고서,윤명순,2017.12.06,NaN


In [5]:
df_list[0].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354 entries, 0 to 353
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       354 non-null    int64 
 1   1       354 non-null    object
 2   2       354 non-null    object
 3   3       354 non-null    object
 4   4       354 non-null    object
 5   5       188 non-null    object
dtypes: int64(1), object(5)
memory usage: 16.7+ KB


# drop column

* 지놈앤컴퍼니 데이터 없음

In [6]:
# df_list

In [7]:
df_list[90]

""


In [8]:
df_list1=[]
for df in df_list:
    if len(df) >= 1:
        df_i = df.iloc[:,[1,2,4]]
        df_i.columns = ['corp_name','report_nm','rcept_dt']
        # df_i['rcept_dt'] = pd.to_datetime(df_i['rcept_dt'])
        df_list1.append(df_i)
    else:
        # df_list1.append(df)
        continue

In [9]:
len(df_list1)

125

## corp_name

In [10]:
df_list1[50]

,corp_name,report_nm,rcept_dt
0,코 쎌바이오텍,주주명부폐쇄기간또는기준일설정,2017.12.15
1,코 쎌바이오텍,분기보고서 (2017.09),2017.11.14
2,코 쎌바이오텍,기업설명회(IR)개최,2017.11.07
3,코 쎌바이오텍,연결재무제표기준영업(잠정)실적(공정공시),2017.11.07
4,코 쎌바이오텍,기타경영사항(특허권취득)(자율공시),2017.10.26
...,...,...,...
184,코 쎌바이오텍,감사보고서제출,2010.03.11
185,코 쎌바이오텍,[기재정정]매출액또는손익구조30%(대규모법인은15%)이상변동 (정정공시),2010.03.11
186,코 쎌바이오텍,주주총회소집공고,2010.03.04
187,코 쎌바이오텍,[기재정정]현금ㆍ현물배당결정 (정정공시),2010.02.25


In [11]:
corp=[]
for df in df_list1:
    tmp = []
    for txt in df['corp_name']:
        # print(txt)
        name = txt.split()[1]
        # print(name)
        tmp.append(name)
    corp.append(tmp)

In [12]:
len(corp)

125

In [13]:
df_list2 = []
for i,df in enumerate(df_list1):
    df['corp_name'] = corp[i]
    df_i = df.sort_values(by=['rcept_dt'], ascending=True)
    df_list2.append(df_i)

In [14]:
df_list2[0].head(2)

,corp_name,report_nm,rcept_dt
353,CMG제약,소송등의판결ㆍ결정,2010.01.18
352,CMG제약,전환가액의조정,2010.01.28


# drop

In [15]:
drop_df = pd.DataFrame(columns = ['df', 'row' ])
drop_df

,df,row


In [16]:
for i, df in enumerate(df_list2):
    for j,title in enumerate(df['report_nm']):
        if ("단일판매" in title) or ("보고서" in title) or("주주총회" in title) or("참고서류" in title):
            # print(1)
            drop_df = drop_df.append({'df':i,'row':j}, ignore_index=True)

In [17]:
drop_df['row'] = drop_df['row'].astype('int')

In [18]:
drop_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17422 entries, 0 to 17421
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   df      17422 non-null  object
 1   row     17422 non-null  int32 
dtypes: int32(1), object(1)
memory usage: 204.3+ KB


In [19]:
len(df_list2[0])

354

In [20]:
df_list3 = []
for i, df in enumerate(df_list2):
    tmp_list =[]
    tmp=drop_df[drop_df['df']==i]['row']
    # tmp_list.append(tmp)
    df = df.drop(tmp.values, axis = 0)
    df_list3.append(df)

In [21]:
len(df_list3[0])

166

# 년도별  df 분리

In [22]:
df_list3[0].head(2)

,corp_name,report_nm,rcept_dt
353,CMG제약,소송등의판결ㆍ결정,2010.01.18
346,CMG제약,주요사항보고서(신주인수권부사채권발행결정),2010.02.24


In [23]:
df_list3[0].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 166 entries, 353 to 0
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   corp_name  166 non-null    object
 1   report_nm  166 non-null    object
 2   rcept_dt   166 non-null    object
dtypes: object(3)
memory usage: 5.2+ KB


In [24]:
year=[]
month=[]
for df in df_list3:
    tmp_y = []
    tmp_m = []
    for txt in df['rcept_dt']:
        # print(txt)
        y = txt.split('.')[0]
        m = txt.split('.')[1]
        # print(name)
        tmp_y.append(y)
        tmp_m.append(m)
    year.append(tmp_y)
    month.append(tmp_m)
    # corp.append(tmp)

In [25]:
# corp[0]

In [26]:
df_list3[0]

,corp_name,report_nm,rcept_dt
353,CMG제약,소송등의판결ㆍ결정,2010.01.18
346,CMG제약,주요사항보고서(신주인수권부사채권발행결정),2010.02.24
345,CMG제약,소액공모공시서류(채무증권),2010.02.25
339,CMG제약,주주총회소집결의,2010.03.08
338,CMG제약,[기재정정]의결권대리행사권유참고서류,2010.03.12
...,...,...,...
7,CMG제약,조회공시요구(현저한시황변동)에대한답변(중요정보없음),2017.11.09
5,CMG제약,임원ㆍ주요주주특정증권등소유상황보고서,2017.11.29
2,CMG제약,임원ㆍ주요주주특정증권등소유상황보고서,2017.12.06
1,CMG제약,임원ㆍ주요주주특정증권등소유상황보고서,2017.12.06


In [27]:
for i,df in enumerate(df_list3):
    df['year']  = year[i]
    df['month']=  month[i]

In [28]:
df_list3[0].head(2)

,corp_name,report_nm,rcept_dt,year,month
353,CMG제약,소송등의판결ㆍ결정,2010.01.18,2010,01
346,CMG제약,주요사항보고서(신주인수권부사채권발행결정),2010.02.24,2010,02


In [29]:
df_2010 =[]
for df in df_list3:
    tmp = df[df['year']=='2010']
    df_2010.append(tmp)

In [30]:
df_2011 =[]
for df in df_list3:
    tmp = df[df['year']=='2011']
    df_2011.append(tmp)

In [31]:
df_2012 =[]
for df in df_list3:
    tmp = df[df['year']=='2012']
    df_2012.append(tmp)

In [32]:
df_2013 =[]
for df in df_list3:
    tmp = df[df['year']=='2013']
    df_2013.append(tmp)

In [33]:
df_2014 =[]
for df in df_list3:
    tmp = df[df['year']=='2014']
    df_2014.append(tmp)

In [34]:
df_2015 =[]
for df in df_list3:
    tmp = df[df['year']=='2015']
    df_2015.append(tmp)

In [35]:
df_2016 =[]
for df in df_list3:
    tmp = df[df['year']=='2016']
    df_2016.append(tmp)

In [36]:
df_2017 =[]
for df in df_list3:
    tmp = df[df['year']=='2017']
    df_2017.append(tmp)

In [37]:
df_2010 = pd.concat(df_2010)
df_2011 = pd.concat(df_2011)
df_2012 = pd.concat(df_2012)
df_2013 = pd.concat(df_2013)
df_2014 = pd.concat(df_2014)
df_2015 = pd.concat(df_2015)
df_2016 = pd.concat(df_2016)
df_2017 = pd.concat(df_2017)

In [38]:
df_2010.tail()

,corp_name,report_nm,rcept_dt,year,month
384,휴온스글로벌,임원ㆍ주요주주특정증권등소유상황보고서,2010.11.08,2010,11
383,휴온스글로벌,분기보고서 (2010.09),2010.11.15,2010,11
382,휴온스글로벌,임원ㆍ주요주주특정증권등소유상황보고서,2010.11.29,2010,11
381,휴온스글로벌,주식등의대량보유상황보고서(일반),2010.11.29,2010,11
377,휴온스글로벌,자기주식처분결과보고서,2010.12.15,2010,12


# positive(+1)
*  DataFrame.insert( loc, column, value)

<pre>
* 기업설명회
신규시설투자
취득
취득결정
유상증자
무상증자
단기차입금감소결정
권리락(무상증자)
발행조건확정
증권발행결과
배당결정
특허권
실적
승인
주식분할
정지해제
대표이사변경

## 칼럼추가

In [ ]:
#!!!!! 키워드 세밀화

In [40]:
df_2010.insert( 5, 'pos', 0)
df_2011.insert( 5, 'pos', 0)
df_2012.insert( 5, 'pos', 0)
df_2013.insert( 5, 'pos', 0)
df_2014.insert( 5, 'pos', 0)
df_2015.insert( 5, 'pos', 0)
df_2016.insert( 5, 'pos', 0)
df_2017.insert( 5, 'pos', 0)

In [41]:
df_2010[:2]

,corp_name,report_nm,rcept_dt,year,month,pos
353,CMG제약,소송등의판결ㆍ결정,2010.01.18,2010,01,0
346,CMG제약,주요사항보고서(신주인수권부사채권발행결정),2010.02.24,2010,02,0


In [42]:
for j,note in enumerate(df_2010['report_nm']):
    if ("기업설명회" in note) or ("신규시설투자" in note) or("취득" in note) or("취득결정" in note) or \
    ("유상증자" in note) or ("무상증자" in note) or ("단기차입금감소결정" in note) or("권리락" in note) \
    or("발행조건확정" in note) or ("증권발행결과" in note) or ("배당결정" in note) or("특허권" in note) \
    or("실적" in note) or("승인" in note) or ("주식분할" in note) or("정지해제" in note) or \
    ("대표이사변경" in note):
        # print(1)
        df.loc[j,'positive']  = 1

In [45]:
df_2010[df_2010['pos']==1]

,corp_name,report_nm,rcept_dt,year,month,pos
